In [3]:
# %load ~/ipyhead
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns


We're going to load quantification data from Kallisto. We used the index: `Homo_sapiens.GRCh38.cdna.all.kallisto.idx`

Per http://f1000research.com/articles/4-1521/v2 (https://github.com/mikelove/tximport), we should use TPM (not estimated counts, which are sensitive to transcript length) and aggregate to gene level.

Here is how to annotate with gene names in Python: https://github.com/hammerlab/cohorts/blob/master/cohorts/load.py#L797-L801

We made the index by downloading from http://ftp.ensembl.org/pub/release-79/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz , which is the same as what's suggested on kallisto website. So use release 79.

Though note that the newest release is release 85, from ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/README_ensembl:

```
#tax_id	ncbi_release	ncbi_assembly	ensembl_release	ensembl_assembly	date_compared
9606	Homo sapiens Annotation Release 108	GRCh38.p7	85	GRCh38.p7	20160801
```

And http://ftp.ensembl.org/pub/release-85/fasta/homo_sapiens/cdna/ has different files.

In [6]:
df = pd.read_csv('output/ERR431617/abundance.tsv', sep='\t')
df.head()

,target_id,length,eff_length,est_counts,tpm
0,ENST00000415118,8,5.08333,0.0,0.00000
1,ENST00000448914,13,8.66667,0.0,0.00000
2,ENST00000434970,9,5.69231,0.0,0.00000
3,ENST00000390577,37,21.37140,1.0,6.44565
4,ENST00000437320,19,11.90480,0.0,0.00000


In [5]:
!pip install pyensembl

    100% |████████████████████████████████| 61kB 1.8MB/s 
    100% |████████████████████████████████| 14.3MB 68kB/s 
    100% |████████████████████████████████| 81kB 4.6MB/s 
  Running setup.py bdist_wheel for pyensembl ... - \ done
  Stored in directory: /home/maxim/.cache/pip/wheels/04/69/38/f675315f843ae0d30475fbbdd865622995dc64ae7adee83a3f
  Running setup.py bdist_wheel for datacache ... - \ done
  Stored in directory: /home/maxim/.cache/pip/wheels/f6/37/55/ba59c1f1e2c7df84eca3df207aba1f9ed385bc4d2f674db8d2
  Running setup.py bdist_wheel for memoized-property ... - done
  Stored in directory: /home/maxim/.cache/pip/wheels/09/42/c6/fa081008bb537f37b06168cb296c94d70f836c49d1f52c78ba
  Running setup.py bdist_wheel for gtfparse ... - \ done
  Stored in directory: /home/maxim/.cache/pip/wheels/ed/90/50/007c4aa4ae0e50fffec744aa3357566eb834fae066651737b9
  Running setup.py bdist_wheel for serializable ... - done
  Stored in directory: /home/maxim/.cache/pip/wheels/24/e1/e2

In [7]:
!pyensembl install --release 79 --species human

-- Running 'install' for EnsemblRelease(release=79, species='homo_sapiens')
INFO:root:Fetching /home/maxim/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.79.gtf.gz from URL ftp://ftp.ensembl.org/pub/release-79/gtf/homo_sapiens/Homo_sapiens.GRCh38.79.gtf.gz
INFO:root:Fetching /home/maxim/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL ftp://ftp.ensembl.org/pub/release-79/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
INFO:root:Fetching /home/maxim/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.pep.all.fa.gz from URL ftp://ftp.ensembl.org/pub/release-79/fasta/homo_sapiens/pep/Homo_sapiens.GRCh38.pep.all.fa.gz
Creating database: /home/maxim/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.79.gtf.db
Reading GTF from /home/maxim/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.79.gtf.gz
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_vers

In [ ]:
from pyensembl import cached_release

In [ ]:
ensembl_release = cached_release(79)
df['sample_id'] = 1
df['gene_name'] = df['target_id'].map(lambda t: ensembl_release.gene_name_of_transcript_id(t))

In [ ]:
# sum counts across genes
df = df.groupby(['sample_id', 'gene_name'])[['est_counts']].sum().reset_index()
# can we just sum() tpm? look at the other github.